<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [ ]:
#default_exp data_loader

In [ ]:
#export
import os
import random
import re
from pathlib import Path
from typing import List

import numpy as np
from scipy.io.wavfile import read
import torch
from torch.utils.data import Dataset

from uberduck_ml_dev.models.common import STFT, MelSTFT
from uberduck_ml_dev.text.util import text_to_sequence
from uberduck_ml_dev.utils import load_filepaths_and_text


In [ ]:
?text_to_sequence

In [ ]:
# export

def _orig_to_dense_speaker_id(speaker_ids):
    speaker_ids = sorted(list(set(speaker_ids)))
    return {
        orig: idx for orig, idx in zip(speaker_ids, range(len(speaker_ids)))
    }


class TextMelDataset(Dataset):
    def __init__(
        self,
        dataset_path: str,
        audiopaths_and_text: str,
        text_cleaners: List[str],
        n_mel_channels: int,
        sample_rate: int,
        mel_fmin: float,
        mel_fmax: float,
        filter_length: int,
        hop_length: int,
        win_length: int,
        max_wav_value: float = 32768.0,
        include_f0: bool = False,
        debug: bool = False,
        debug_dataset_size: int = None,
    ):
        super().__init__()
        if include_f0:
            raise NotImplemented
        path = str(Path(dataset_path) / audiopaths_and_text)
        self.dataset_path = dataset_path
        self.audiopaths_and_text = load_filepaths_and_text(path)
        self.text_cleaners = text_cleaners
        self.stft = MelSTFT(
            filter_length=filter_length,
            hop_length=hop_length,
            win_length=win_length,
            n_mel_channels=n_mel_channels,
            sampling_rate=sample_rate,
            mel_fmin=mel_fmin,
            mel_fmax=mel_fmax,
        )
        self.max_wav_value = max_wav_value
        self.include_f0 = include_f0
        # speaker id lookup table
        speaker_ids =[i[2] for i in self.audiopaths_and_text]
        self._speaker_id_map = _orig_to_dense_speaker_id(speaker_ids)
        self.debug = debug
        self.debug_dataset_size = debug_dataset_size
        

    def _get_data(self, audiopath_and_text):
        path, transcription, speaker_id = audiopath_and_text
        speaker_id = self._speaker_id_map[speaker_id]
        path = Path(self.dataset_path) / path
        sample_rate, wav_data = read(path)
        text_sequence = torch.LongTensor(text_to_sequence(transcription, self.text_cleaners))
        audio = torch.FloatTensor(wav_data)
        audio_norm = audio / self.max_wav_value
        audio_norm = audio_norm.unsqueeze(0)
        melspec = self.stft.mel_spectrogram(audio_norm)
        melspec = torch.squeeze(melspec, 0)
        return (text_sequence, melspec, speaker_id)
        

    def __getitem__(self, idx):
        """Return"""
        return self._get_data(self.audiopaths_and_text[idx])

    def __len__(self):
        if self.debug and self.debug_dataset_size:
            return min(self.debug_dataset_size, len(self.audiopaths_and_text))
        return len(self.audiopaths_and_text)

In [ ]:
# export


class TextMelCollate:
    def __init__(self, n_frames_per_step: int = 1, include_f0: bool = False):
        self.n_frames_per_step = n_frames_per_step
        self.include_f0 = include_f0

    def __call__(self, batch):
        """Collate's training batch from normalized text and mel-spectrogram
        PARAMS
        ------
        batch: [text_normalized, mel_normalized, speaker_id]
        """
        # Right zero-pad all one-hot text sequences to max input length
        input_lengths, ids_sorted_decreasing = torch.sort(
            torch.LongTensor([len(x[0]) for x in batch]), dim=0, descending=True
        )
        max_input_len = input_lengths[0]

        text_padded = torch.LongTensor(len(batch), max_input_len)
        text_padded.zero_()
        for i in range(len(ids_sorted_decreasing)):
            text = batch[ids_sorted_decreasing[i]][0]
            text_padded[i, : text.size(0)] = text

        # Right zero-pad mel-spec
        num_mels = batch[0][1].size(0)
        max_target_len = max([x[1].size(1) for x in batch])
        if max_target_len % self.n_frames_per_step != 0:
            max_target_len += (
                self.n_frames_per_step - max_target_len % self.n_frames_per_step
            )
            assert max_target_len % self.n_frames_per_step == 0

        # include mel padded, gate padded and speaker ids
        mel_padded = torch.FloatTensor(len(batch), num_mels, max_target_len)
        mel_padded.zero_()
        gate_padded = torch.FloatTensor(len(batch), max_target_len)
        gate_padded.zero_()
        output_lengths = torch.LongTensor(len(batch))
        speaker_ids = torch.LongTensor(len(batch))
        if self.include_f0:
            f0_padded = torch.FloatTensor(len(batch), 1, max_target_len)
            f0_padded.zero_()

        for i in range(len(ids_sorted_decreasing)):
            mel = batch[ids_sorted_decreasing[i]][1]
            mel_padded[i, :, : mel.size(1)] = mel
            gate_padded[i, mel.size(1) - 1 :] = 1
            output_lengths[i] = mel.size(1)
            speaker_ids[i] = batch[ids_sorted_decreasing[i]][2]
            if self.include_f0:
                f0 = batch[ids_sorted_decreasing[i]][3]
                f0_padded[i, :, : f0.size(1)] = f0

        # NOTE(zach): would model_inputs be better as a namedtuple or dataclass?
        if self.include_f0:
            model_inputs = (
                text_padded,
                input_lengths,
                mel_padded,
                gate_padded,
                output_lengths,
                speaker_ids,
                f0_padded,
            )
        else:
            model_inputs = (
                text_padded,
                input_lengths,
                mel_padded,
                gate_padded,
                output_lengths,
                speaker_ids,
            )

        return model_inputs

In [ ]:
_orig_to_dense_speaker_id([4, 2, 9, 3, 9])

{2: 0, 3: 1, 4: 2, 9: 3}

In [ ]:
ds = TextMelDataset(
    "./test/fixtures",
     "val.txt",
    ["english_cleaners"],
    80,
    22050,
    0,
    8000,
    1024,
    256,
    1024,
    debug=True,
    debug_dataset_size=12,
)
len(ds)

1

In [ ]:
from torch.utils.data import DataLoader
collate_fn = TextMelCollate()
dl = DataLoader(ds, 12, collate_fn=collate_fn)
for i, batch in enumerate(dl):
    print(i)
    print(batch)

0
(tensor([[97, 79, 86, 86,  3,  8, 99, 89, 95,  1, 92, 79,  4,  8, 99, 89, 95,  1,
         92, 79,  8, 88, 89, 94,  8, 92, 79, 75, 86, 86, 99,  8, 75,  8, 90, 79,
         92, 93, 89, 88,  4,  8, 99, 89, 95,  1, 92, 79,  8, 87, 89, 92, 79,  8,
         89, 80,  8, 75, 88,  8, 75, 88, 94, 82, 92, 89, 90, 89, 87, 89, 92, 90,
         82, 83, 77,  8, 78, 89, 81,  8, 94, 82, 83, 88, 81,  4]]), tensor([86]), tensor([[[ -7.4811,  -7.1174,  -6.2034,  ...,  -6.8980,  -6.4200,  -6.8893],
         [ -7.6766,  -7.0730,  -6.0029,  ...,  -6.4762,  -6.0089,  -6.4041],
         [ -6.9245,  -6.7840,  -6.4049,  ...,  -6.2306,  -6.1970,  -6.6943],
         ...,
         [ -9.6343,  -9.4932,  -9.6086,  ...,  -9.2852,  -9.5855,  -9.9288],
         [ -9.5884,  -9.4754,  -9.6879,  ...,  -9.2717,  -9.2050,  -9.6523],
         [-10.0712,  -9.8129,  -9.7728,  ...,  -9.4478,  -9.2810,  -9.3694]]]), tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0.